In [ ]:
"""下载LLaMA-Factory
"""
!git clone https://www.modelscope.cn/allspace/ChatGLM3_6B_Lora.git

Cloning into 'ChatGLM3_6B_Lora'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (23/23), 26.00 KiB | 178.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
"""安装依赖
"""
!pip install einops
!pip install transformers==4.34.0

In [ ]:
"""查看模型效果
"""


import os
from threading import Thread

import torch
from transformers.generation.streamers import TextIteratorStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

torch.set_default_device("cuda")

model_name = "/content/ChatGLM3_6B_Lora"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


def stream(prompt):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, return_attention_mask=True)
    streamer = TextIteratorStreamer(
        tokenizer,
        decode_kwargs={"skip_special_tokens": True})
    Thread(
        target=model.generate, kwargs=dict(
            inputs, streamer=streamer,
            max_new_tokens=128)
    ).start()

    first = True
    for text in streamer:
        if first and text:
            first = False
            continue

        if not text:
            continue
        if tokenizer.eos_token in text:
            break

        yield text


def main():
    welcome_prompt = "欢迎使用 法律AI 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序"
    print(welcome_prompt)
    while True:
        query = input("\n用户：")
        if query.strip() == "stop":
            break
        if query.strip() == "clear":
            os.system("clr")
            print(welcome_prompt)
            continue
        print("\n法律AI：", end="")
        for text in stream(query):
            print(text, end="", flush=True)
        print("")



main()